<a href="https://colab.research.google.com/github/daniel9511/Curso-Spark/blob/main/ejercicio_funciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalar SDK java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# descargar spark
!wget -q https://archive.apache.org/dist/spark/spark-3.2.3/spark-3.2.3-bin-hadoop3.2.tgz

# descomprimir la version de spark
!tar xf spark-3.2.3-bin-hadoop3.2.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.3-bin-hadoop3.2"

# descargar findspark
!pip install -q findspark

# descargar pyspark
!pip install -q pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
# importar findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [5]:
df = spark.read.option('header', True).csv('./data/Case.csv')
df.show()

+--------+--------+---------------+-----+--------------------+---------+---------+----------+
| case_id|province|           city|group|      infection_case|confirmed| latitude| longitude|
+--------+--------+---------------+-----+--------------------+---------+---------+----------+
| 1000001|   Seoul|     Yongsan-gu| TRUE|       Itaewon Clubs|      139|37.538621|126.992652|
| 1000002|   Seoul|      Gwanak-gu| TRUE|             Richway|      119| 37.48208|126.901384|
| 1000003|   Seoul|        Guro-gu| TRUE| Guro-gu Call Center|       95|37.508163|126.884387|
| 1000004|   Seoul|   Yangcheon-gu| TRUE|Yangcheon Table T...|       43|37.546061|126.874209|
| 1000005|   Seoul|      Dobong-gu| TRUE|     Day Care Center|       43|37.679422|127.044374|
| 1000006|   Seoul|        Guro-gu| TRUE|Manmin Central Ch...|       41|37.481059|126.894343|
| 1000007|   Seoul|from other city| TRUE|SMR Newly Planted...|       36|        -|         -|
| 1000008|   Seoul|  Dongdaemun-gu| TRUE|       Dongan Churc

In [41]:
from pyspark.sql.functions import col

df_columns = df.select([col('province'), col('city'), col('confirmed')])

df_columns.show()

df_order1 = df_columns.withColumn('confirmed', col('confirmed').cast('int'))

df_order1.printSchema()

df_groupby = df_order1.groupBy(['province', 'city']).sum('confirmed')

df_groupby.orderBy(col('sum(confirmed)'), ascending=False).show(3)




+--------+---------------+---------+
|province|           city|confirmed|
+--------+---------------+---------+
|   Seoul|     Yongsan-gu|      139|
|   Seoul|      Gwanak-gu|      119|
|   Seoul|        Guro-gu|       95|
|   Seoul|   Yangcheon-gu|       43|
|   Seoul|      Dobong-gu|       43|
|   Seoul|        Guro-gu|       41|
|   Seoul|from other city|       36|
|   Seoul|  Dongdaemun-gu|       17|
|   Seoul|from other city|       25|
|   Seoul|      Gwanak-gu|       30|
|   Seoul|   Eunpyeong-gu|       14|
|   Seoul|   Seongdong-gu|       13|
|   Seoul|      Jongno-gu|       10|
|   Seoul|     Gangnam-gu|        7|
|   Seoul|        Jung-gu|        7|
|   Seoul|   Seodaemun-gu|        5|
|   Seoul|      Jongno-gu|        7|
|   Seoul|     Gangnam-gu|        6|
|   Seoul|from other city|        1|
|   Seoul|   Geumcheon-gu|        6|
+--------+---------------+---------+
only showing top 20 rows

root
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- 

In [47]:
df1 = spark.read.option('header', True).csv('./data/PatientInfo.csv')
df1.show()

df1_infected_by = df1.select(col('infected_by')).na.drop(subset=['infected_by'])

df1_infected_by.show()


+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|released_date|deceased_date|   state|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+-------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul|  Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|    2020-01-23|   2020-02-05|         null|released|
|1000000002|  male|30s|  Korea|   Seoul| Jungnang-gu|     overseas inflow|       null|            31|              null|    2020-01-30|   2020-03-02|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|   Jongno-gu|contact with patient| 2002000001|            17|              null|    2020-01-30|

In [53]:
dfb = df1.filter(col('sex') == 'female').drop('released_date', 'deceased_date')
dfb.show()

+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+--------+
|patient_id|   sex|age|country|province|        city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|   state|
+----------+------+---+-------+--------+------------+--------------------+-----------+--------------+------------------+--------------+--------+
|1000000005|female|20s|  Korea|   Seoul| Seongbuk-gu|contact with patient| 1000000002|             2|              null|    2020-01-31|released|
|1000000006|female|50s|  Korea|   Seoul|   Jongno-gu|contact with patient| 1000000003|            43|              null|    2020-01-31|released|
|1000000010|female|60s|  Korea|   Seoul| Seongbuk-gu|contact with patient| 1000000003|             6|              null|    2020-02-05|released|
|1000000011|female|50s|  China|   Seoul|Seodaemun-gu|     overseas inflow|       null|            23|              null|    2020-0